In [1]:
import csv
import networkx as nx
import numpy as np
from sklearn.linear_model import LogisticRegression
import community as community_louvain
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from random import randint

### Load the retweet network as a directed graph

In [2]:
G = nx.read_weighted_edgelist("data/retweet_weighted.edgelist", create_using=nx.DiGraph(), nodetype=int)

print("Number of nodes:", G.number_of_nodes())
print("Number of edges:", G.number_of_edges())

Number of nodes: 784898
Number of edges: 7401920


### Store the ID of the user that posted each message, and initialize for each user a 15-dimensional vector that will store the number of messages of each class posted by the user

In [3]:
posted_by = dict()
posts_per_class = dict()
users = list()
with open('data/posts.tsv', 'r') as f:
    for line in f:
        t = line.split('\t')
        posted_by[int(t[0])] = int(t[1])
        posts_per_class[int(t[1])] = np.zeros(15)
        users.append(int(t[1]))
users = set(users)

### Read training data. Given a message posted by user A that belongs to class B, increase the number of posts of class B posted by user A by 1 

In [4]:
train_index = list()
y_train = list()    
with open('data/train.csv', 'r') as f:
    for line in f:
        t = line.split(',')
        train_index.append(int(t[0]))
        y_train.append(int(t[1]))
        posts_per_class[posted_by[int(t[0])]][int(t[1][:-1])] += 1

### Read test data

In [5]:
test_index = list()  
with open('data/test.csv', 'r') as f:
    for line in f:
        t = line.split(',')
        test_index.append(int(t[0]))

## Community Detection

In [6]:
G_un = G.copy().to_undirected()
for node in G:
    for ngbr in nx.neighbors(G, node):
        if node in nx.neighbors(G, ngbr):
            G_un.edges[node, ngbr]['weight'] = (
                G.edges[node, ngbr]['weight'] + G.edges[ngbr, node]['weight']
            )

In [7]:
print("Number of edges:", G.number_of_edges())
print("Number of undirected edges:", G_un.number_of_edges())

Number of edges: 7401920
Number of undirected edges: 7383985


In [103]:
initial_partition = dict()
for user in G.nodes():
    initial_partition[user] = -1
    has_succ = False
    has_pred = False
    succ_posts_per_class = np.zeros((1, 15))
    pred_posts_per_class = np.zeros((1, 15))
    for successor in G.successors(user):
        if successor in posts_per_class:
            has_succ = True
            succ_posts_per_class[0,:15] += posts_per_class[successor]
    for predecessor in G.predecessors(user):
        if predecessor in posts_per_class:
            has_pred = True
            pred_posts_per_class[0,:15] += posts_per_class[predecessor]

    if user in posts_per_class:
        initial_partition[user] = posts_per_class[user].tolist().index(max(posts_per_class[user]))
    elif has_pred:
        maxValue = np.max(pred_posts_per_class)
        index_of_maximum = (np.where(pred_posts_per_class[0] == maxValue)[0])[0]
        if(user%2 == 0):
            initial_partition[user] = index_of_maximum+15
        else:
            initial_partition[user] = index_of_maximum+30
    elif has_succ:
        maxValue = np.max(succ_posts_per_class)
        index_of_maximum = (np.where(succ_posts_per_class[0] == maxValue)[0])[0]
        if(user%2 == 0):
            initial_partition[user] = index_of_maximum+45
        else:
            initial_partition[user] = index_of_maximum+60


In [104]:
for comm in set(initial_partition.values()):
    times = list(initial_partition.values()).count(comm)
    print('{} has occurred {} times'.format(comm, times))

0 has occurred 6994 times
1 has occurred 458 times
2 has occurred 1855 times
3 has occurred 811 times
4 has occurred 829 times
5 has occurred 105 times
6 has occurred 78 times
7 has occurred 14 times
8 has occurred 75 times
9 has occurred 94 times
10 has occurred 416 times
11 has occurred 230 times
12 has occurred 246 times
13 has occurred 143 times
14 has occurred 292 times
15 has occurred 6842 times
16 has occurred 1137 times
17 has occurred 4217 times
18 has occurred 537 times
19 has occurred 1374 times
20 has occurred 109 times
21 has occurred 189 times
22 has occurred 14 times
23 has occurred 59 times
24 has occurred 256 times
25 has occurred 609 times
26 has occurred 390 times
27 has occurred 482 times
28 has occurred 103 times
29 has occurred 212 times
30 has occurred 6938 times
31 has occurred 1156 times
32 has occurred 4097 times
33 has occurred 533 times
34 has occurred 1369 times
35 has occurred 100 times
36 has occurred 194 times
37 has occurred 12 times
38 has occurred 45 

In [8]:
# compute the best partition
partition = community_louvain.best_partition(G_un,resolution = 0.8)

In [9]:
for comm in set(partition.values()):
    times = list(partition.values()).count(comm)
    print('{} has occurred {} times'.format(comm, times))


0 has occurred 88338 times
1 has occurred 92569 times
2 has occurred 117290 times
3 has occurred 127860 times
4 has occurred 84665 times
5 has occurred 145582 times
6 has occurred 17908 times
7 has occurred 21268 times
8 has occurred 22571 times
9 has occurred 4716 times
10 has occurred 6870 times
11 has occurred 3887 times
12 has occurred 15943 times
13 has occurred 762 times
14 has occurred 22999 times
15 has occurred 610 times
16 has occurred 1 times
17 has occurred 3029 times
18 has occurred 5 times
19 has occurred 2 times
20 has occurred 1 times
21 has occurred 45 times
22 has occurred 385 times
23 has occurred 562 times
24 has occurred 426 times
25 has occurred 1 times
26 has occurred 1 times
27 has occurred 3 times
28 has occurred 2 times
29 has occurred 800 times
30 has occurred 4 times
31 has occurred 2 times
32 has occurred 2 times
33 has occurred 653 times
34 has occurred 4 times
35 has occurred 4 times
36 has occurred 1 times
37 has occurred 6 times
38 has occurred 1 times


340 has occurred 2 times
341 has occurred 2 times
342 has occurred 1 times
343 has occurred 7 times
344 has occurred 3 times
345 has occurred 5 times
346 has occurred 7 times
347 has occurred 2 times
348 has occurred 2 times
349 has occurred 6 times
350 has occurred 1 times
351 has occurred 3 times
352 has occurred 2 times
353 has occurred 1 times
354 has occurred 3 times
355 has occurred 1 times
356 has occurred 2 times
357 has occurred 2 times
358 has occurred 2 times
359 has occurred 2 times
360 has occurred 2 times
361 has occurred 2 times
362 has occurred 2 times
363 has occurred 2 times
364 has occurred 1 times
365 has occurred 1 times
366 has occurred 3 times
367 has occurred 2 times
368 has occurred 2 times
369 has occurred 1 times
370 has occurred 2 times
371 has occurred 3 times
372 has occurred 2 times
373 has occurred 1 times
374 has occurred 1 times
375 has occurred 1 times
376 has occurred 2 times
377 has occurred 1 times
378 has occurred 2 times
379 has occurred 2 times


### Calculate the posibility of each class per community

In [10]:
items_per_comm = np.zeros((len(set(partition.values())), 15))
pos_per_comm = np.zeros((len(set(partition.values())), 15))
for i,idx in enumerate(train_index):
    community = partition[posted_by[idx]]
    items_per_comm[community,:15] += posts_per_class[posted_by[idx]]
pos_per_comm = items_per_comm
for i in range(0,len(pos_per_comm)):   
    if np.sum(items_per_comm[i,:15]) > 0:
        pos_per_comm[i,:15] /= np.sum(items_per_comm[i,:15])

 ### Create the training matrix. Each row corresponds to a message. Use the following 15-dimensional vector of the user that posted the message and concatenate to that vector the following three features:<br/>(1) in-degree of user <br/> (2) out-degree of user <br/> (3) community user belongs to

In [11]:
X_train = np.zeros((len(train_index), 17))
for i,idx in enumerate(train_index):
    for successor in G.successors(posted_by[idx]):
        if partition[successor] == partition[posted_by[idx]]:
            if successor in posts_per_class:
                X_train[i,:15] += posts_per_class[successor]
    
    for predecessor in G.predecessors(posted_by[idx]):
        if partition[predecessor] == partition[posted_by[idx]]:
            if predecessor in posts_per_class:
                X_train[i,:15] += posts_per_class[predecessor]
    if np.sum(X_train[i,:15]) > 0:
        X_train[i,:15] /= np.sum(X_train[i,:15])
    
    X_train[i,15] = G_un.degree(posted_by[idx])
    X_train[i,16] = partition[posted_by[idx]]
X_train_dev, X_test_dev, y_train_dev, y_test_dev = train_test_split(X_train, y_train, test_size=0.2)

### Create the test matrix. Each row corresponds to a message.Use the following 15-dimensional vector of the user that posted the message and concatenate to that vector the following three features: <br/>(1) in-degree of user<br/>(2) out-degree of user <br/> (3) community user belongs to

In [12]:
X_test = np.zeros((len(test_index), 17))
for i,idx in enumerate(test_index):
    for successor in G.successors(posted_by[idx]):
        if partition[successor] == partition[posted_by[idx]]:
            if successor in posts_per_class:
                X_test[i,:15] += posts_per_class[successor]
    
    for predecessor in G.predecessors(posted_by[idx]):
        if partition[predecessor] == partition[posted_by[idx]]:
            if predecessor in posts_per_class:
                X_test[i,:15] += posts_per_class[predecessor]

    if np.sum(X_test[i,:15]) > 0:
        X_test[i,:15] /= np.sum(X_test[i,:15])

    X_test[i,15] = G_un.degree(posted_by[idx])
    X_test[i,16] = partition[posted_by[idx]]

### Use logistic regression to classify the messages of the test set


In [13]:
clf = LogisticRegression(solver='newton-cg', multi_class='auto')
clf.fit(X_train_dev, y_train_dev)
y_pred_lin = clf.predict_proba(X_test_dev)
y_pred_dev_lin = clf.predict_proba(X_test_dev)

/Users/pspyrakis/opt/anaconda3/lib/python3.7/site-packages/sklearn/utils/optimize.py:212: ConvergenceWarning: newton-cg failed to converge. Increase the number of iterations.
  "number of iterations.", ConvergenceWarning)


In [14]:
log = log_loss(y_test_dev,y_pred_dev_lin)
print("log loss lin:", log)

log loss lin: 1.463698147182245


In [15]:
clf = make_pipeline(StandardScaler(), SVC(gamma='auto',probability = True,class_weight='balanced'))
clf.fit(X_train_dev, y_train_dev)
y_pred = clf.predict_proba(X_test_dev)
y_pred_dev_SVC = clf.predict_proba(X_test_dev)

In [16]:
log = log_loss(y_test_dev,y_pred_dev_SVC)
print("log loss SVC:", log)

log loss SVC: 1.5523828361712857


In [17]:
clf = RandomForestClassifier(max_depth=12,criterion = 'entropy')
clf.fit(X_train_dev, y_train_dev)
y_pred = clf.predict_proba(X_test_dev)
y_pred_dev_Forest = clf.predict_proba(X_test_dev)

In [18]:
log = log_loss(y_test_dev,y_pred_dev_Forest)
print("log loss forest:", log)

log loss forest: 1.2279791094057024


In [19]:
clf = clf = make_pipeline(StandardScaler(),
                          SGDClassifier(tol=1e-6,
                                        loss = 'log',
                                        eta0 = 0.0001,
                                        learning_rate = 'adaptive'))
clf.fit(X_train_dev, y_train_dev)
y_pred = clf.predict_proba(X_test_dev)
y_pred_dev_SGD = clf.predict_proba(X_test_dev)

In [20]:
log = log_loss(y_test_dev,y_pred_dev_SGD)
print("log loss SGD:", log)

log loss SGD: 1.4543472769046584


### Write predictions to a file

In [146]:
with open('best_so_far.csv', 'w') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    lst = ['id']
    for i in range(15):
        lst.append('class_'+str(i))
    writer.writerow(lst)
    for i,idx in enumerate(test_index):
        lst = y_pred[i,:].tolist()
        lst.insert(0, idx)
        writer.writerow(lst)

### Community Detection2
#### Girvan Newman method which takes a lot of time

In [ ]:
def edge_to_remove(graph):
  G_dict = nx.edge_betweenness_centrality(graph)
  edge = ()

  # extract the edge with highest edge betweenness centrality score
  for key, value in sorted(G_dict.items(), key=lambda item: item[1], reverse = True):
      edge = key
      break

  return edge

In [ ]:
def girvan_newman(graph):
    # find number of connected components
    sg = nx.connected_components(graph)
    sg_count = nx.number_connected_components(graph)

    while(sg_count == 1):
        graph.remove_edge(edge_to_remove(graph)[0], edge_to_remove(graph)[1])
        sg = nx.connected_components(graph)
        sg_count = nx.number_connected_components(graph)

    return sg

In [ ]:
# find communities in the graph
c = girvan_newman(G_un.copy())

# find the nodes forming the communities
node_groups = []

for i in c:
  node_groups.append(list(i))